# 지도학습(4) - 회귀모델 성능 측정 지표

회귀분석을 통해 예측 모델을 만들고 해당 모델의 성능을 파악 하기 위해 제공되는 사이킷런의 성능 지표 모듈

## #01. 작업준비

### 패키지 가져오기 


In [1]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
import helper


from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_error

import numpy as np

## #02. 데이터 가져오기 


In [2]:
origin= pd.read_excel('https://data.hossam.kr/E04/cars.xlsx')
origin.head()

,speed,dist
0,4,2
1,4,10
2,7,4
3,7,22
4,8,16


In [3]:
poly = PolynomialFeatures(degree=2,include_bias=False)
fit = poly.fit_transform(origin[['speed']])
x = pd.DataFrame(fit,columns=poly.get_feature_names_out())
x.head()

,speed,speed^2
0,4.0,16.0
1,4.0,16.0
2,7.0,49.0
3,7.0,49.0
4,8.0,64.0


In [4]:
y= origin[['dist']]

In [5]:
model = LinearRegression()

In [6]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [7]:
fit = model.fit(x_train[['speed']],y_train[['dist']])

In [8]:

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
x_train.head()

(35, 2)
(35, 1)
(15, 2)
(15, 1)


,speed,speed^2
44,23.0,529.0
5,9.0,81.0
12,12.0,144.0
19,14.0,196.0
8,10.0,100.0


In [9]:
model = LinearRegression()
fit = model.fit(x_train,y_train)

## 직선의 기울기 
coef = fit.coef_



## 절편 
intercept = fit.intercept_

print ('기울기: ', coef)

print('절편 : ',intercept )


rscore = fit.score(x_train,y_train)
rscore_test = fit.score(x_test,y_test)
print('훈련 데이터 설명력 : ',rscore)
print('검증 데이터 설명력 : ',rscore_test)

기울기:  [[1.37851995 0.0808227 ]]
절편 :  [-0.75834708]
훈련 데이터 설명력 :  0.5964088514844106
검증 데이터 설명력 :  0.7668043398633401


In [10]:
y_pred_train = fit.predict(x_train)
y_pred_train = y_pred_train.reshape(-1)
y_pred_train

array([73.70281978, 18.19497114, 27.42236105, 34.38218132, 21.10912238,
       54.61052648, 41.98858319, 12.85160485, 34.38218132, 34.38218132,
        6.04889592, 27.42236105, 30.82144849, 38.10455956, 46.03425222,
       59.1411317 , 41.98858319, 30.82144849, 78.8800066 , 50.24156665,
       27.42236105,  6.04889592, 59.1411317 , 21.10912238, 24.18491902,
       50.24156665, 27.42236105, 78.8800066 , 68.68727835, 30.82144849,
       46.03425222, 12.85160485, 50.24156665, 59.1411317 , 50.24156665])

In [11]:
y_test_pred = fit.predict(x_test)
y_test_pred = y_test_pred.reshape(-1)
y_test_pred

array([84.21883882, 54.61052648, 59.1411317 , 54.61052648, 15.4424653 ,
       21.10912238, 78.8800066 , 24.18491902, 78.8800066 , 38.10455956,
       38.10455956, 30.82144849, 34.38218132, 59.1411317 , 46.03425222])

## #03. 회귀분석 모델의 성능평가

회귀분석 모델의 평가를 위한 지표는 관측치와 회귀 예측값의 차이를 기반으로 한다.

|구분|설명|
|--|--|
|에러율|낮을수록 좋다.(0에 가까울 수록 좋음) |
|설명력|높을수록 좋다.(1에 가까울 수록 좋음) |

### 1) $R^2$ (결정계수)

회귀분석에서 가장 많이 채택되는 설명력 값

기본적으로 모델의 학습 결과를 갖고 있는 fit 객체의 score() 매서드로 알 수 있음




In [12]:
print('훈련 데이터 설명력 : ',rscore)
print('검증 데이터 설명력 : ',rscore_test)

훈련 데이터 설명력 :  0.5964088514844106
검증 데이터 설명력 :  0.7668043398633401


sklearn.metrics 에서 r2_score 매서드를 통해서도 조회 할 수 있다.


In [13]:

print('훈련 데이터 설명력 : ',r2_score(y_train,y_pred_train))

print('검증 데이터 설명력 : ',r2_score(y_test,y_test_pred))


훈련 데이터 설명력 :  0.5964088514844106
검증 데이터 설명력 :  0.7668043398633401


### 2) 에러율

### $MAE$ (Mean Absoulte Error) 평균 절대 오차

모델의 예측값과 실제값의 차이를 모두 더하는 개념이다

젛대값을 취하기 떄문에 직관적으로 알 수 있는 지표이다.

이상치에 영향을 받지 않는다.

MAE는 절대값을 취하는 지표기 떄문에 실제보다 작은 값인지, 큰값인지 알 수 없다 .

In [14]:


print('훈련 데이터 MAE : ',mean_absolute_error(y_train,y_pred_train))

print('훈련 데이터 MAE : ',mean_absolute_error(y_test,y_test_pred))

훈련 데이터 MAE :  11.572674137355365
훈련 데이터 MAE :  9.607684246792042


### $MSE$ (Mean Squared Error) 평균 제곱 오차




In [15]:

print('훈련 데이터 MSE : ',mean_squared_error(y_train,y_pred_train))

print('훈련 데이터 MSE : ',mean_squared_error(y_test,y_test_pred))

훈련 데이터 MSE :  250.95462345780382
훈련 데이터 MSE :  142.77171096206865


### $RMSE$ (Root Mean Squared Error) 평균 제곱근 오차


MSE 값에 루트를 씌운다

오류지표를 실제 값과 유사한 단위로 변환하여 해석을 쉽게한다.

In [16]:

print('훈련 데이터 RMSE : ',np.sqrt(mean_squared_error(y_train,y_pred_train)))

print('훈련 데이터 RMSE : ',np.sqrt(mean_squared_error(y_test,y_test_pred)))

훈련 데이터 RMSE :  15.841547382052166
훈련 데이터 RMSE :  11.948711686289391


### $MAPE$ (Mean Absolute Percentage Error) 평균 절대 백분율 오차

MAE를 퍼센트로 변환

MAE와 동일하게 MSE보다 이상치에 민감하여 실제값보다 낮은 값인지 높은 값인지 알 수 없다.

모델에 따른 편향이 있다.

In [17]:

print('훈련 데이터 MAPE : ',np.mean(np.abs((y_train.values - y_pred_train)/y_train.values))*100)

print('검증 데이터 MAPE : ',np.mean(np.abs((y_test.values - y_test_pred)/y_test.values))*100)

훈련 데이터 MAPE :  151.1058859228037
검증 데이터 MAPE :  66.1909931008276


### $MPE$ (Mean Percentage Error) 평균 비율 오차

MAPE 에서 절대값을 제외한 지표 

In [18]:

print('훈련 데이터 MAPE : ',np.mean((y_train.values - y_pred_train)/y_train.values)*100)

print('검증 데이터 MAPE : ',np.mean((y_test.values - y_test_pred)/y_test.values)*100)

훈련 데이터 MAPE :  -109.69911646913451
검증 데이터 MAPE :  -23.067507485789175


## #04. 머신러닝 회귀분석 모듈테스트

### 회귀분석 수행


In [19]:
olsResult = helper.ml_ols(origin,xnames='speed',yname='dist',degree=2,test_size=0.3)

print("계수: ", olsResult.coef)
print("절편: ", olsResult.intercept)

계수:  [[-0.57291577  0.15448622]]
절편:  [10.59889578]


In [20]:
type(origin['dist']) ==pd.Series

True

In [21]:
print("R^2: ", olsResult.trainRegMetric.r2)
print("MAE: ", olsResult.trainRegMetric.mae)
print("MSE: ", olsResult.trainRegMetric.mse)
print("RMSE: ", olsResult.trainRegMetric.rmse)
print("MAPE: ", olsResult.trainRegMetric.mape)
print("MPE: ", olsResult.trainRegMetric.mpe)

R^2:  0.7362907694394939
MAE:  10.49486983811964
MSE:  189.8779652801677
RMSE:  13.779621376517126
MAPE:  32.08240536981032
MPE:  -14.626018915206958


In [22]:
print("R^2: ", olsResult.testRegMetric.r2)
print("MAE: ", olsResult.testRegMetric.mae)
print("MSE: ", olsResult.testRegMetric.mse)
print("RMSE: ", olsResult.testRegMetric.rmse)
print("MAPE: ", olsResult.testRegMetric.mape)
print("MPE: ", olsResult.testRegMetric.mpe)

R^2:  0.36333742876701225
MAE:  12.77297087252875
MSE:  291.8178265389246
RMSE:  17.08267621126516
MAPE:  59.857459284118555
MPE:  -35.04350737847307
